In [1]:
%load_ext autoreload
%autoreload 2
import os
os.chdir('/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference')
print(os.getcwd())

/home/svcl-oowl/brandon/research/CVPR_2021_REFINE/sil_consistent_at_inference


In [2]:
import pprint
import glob
from pathlib import Path
import pickle
import random

import torch
from PIL import Image
import numpy as np
from pytorch3d.renderer import look_at_view_transform, PointLights, TexturesVertex, Textures
import matplotlib.pyplot as plt
import trimesh
import cv2
import pytorch3d
from pytorch3d.io import save_obj

from utils import general_utils
from utils import visualization_tools
from utils.eval_utils import eval_metrics
#from evaluation import compute_iou_2d, compute_iou_2d_given_pose, compute_iou_3d, compute_chamfer_L1

In [3]:
def midpoint(points_list_original):
    # gotcha: need to operate on a copy, or else the underlying data will be changed
    points_list = [points.detach().clone() for points in points_list_original]
    midpoint = points_list[0]
    for point in points_list[1:]:
        midpoint += point
    midpoint = midpoint/(len(points_list))
    return midpoint

In [192]:
def create_hole_in_mesh_old(mesh, face_idx):
    mesh_verts = mesh.verts_packed().detach().clone()
    mesh_faces = mesh.faces_packed().detach().clone()
    mesh_device = mesh_verts.device
    
    face_idx_x = mesh_faces[face_idx][0].item()
    face_idx_y = mesh_faces[face_idx][1].item()
    face_idx_z = mesh_faces[face_idx][2].item()
    vertex_x = mesh_verts[face_idx_x]
    vertex_y = mesh_verts[face_idx_y]
    vertex_z = mesh_verts[face_idx_z]
    vertex_c = midpoint([vertex_x, vertex_y, vertex_z])
    
    # creating 6 new vertices
    vertex_xy_0 = midpoint([vertex_x, vertex_y]).unsqueeze(0)
    vertex_xz_1 = midpoint([vertex_x, vertex_z]).unsqueeze(0)
    vertex_yz_2 = midpoint([vertex_y, vertex_z]).unsqueeze(0)
    vertex_cx_3 = midpoint([vertex_c, vertex_x]).unsqueeze(0)
    vertex_cy_4 = midpoint([vertex_c, vertex_y]).unsqueeze(0)
    vertex_cz_5 = midpoint([vertex_c, vertex_z]).unsqueeze(0)
    new_verts_list = [mesh_verts, vertex_xy_0, vertex_xz_1, vertex_yz_2, vertex_cx_3, vertex_cy_4, vertex_cz_5]
    hole_mesh_verts = torch.cat(new_verts_list, dim=0)
    
    # removing original mesh face, and creating new 9 faces
    new_faces_list = [mesh_faces[np.array([i for i in range(len(mesh_faces)) if i!= face_idx])]]
    new_faces_list.append(torch.tensor([[face_idx_x, 1+len(mesh_verts), 3+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[face_idx_x, 0+len(mesh_verts), 3+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[face_idx_y, 0+len(mesh_verts), 4+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[face_idx_y, 2+len(mesh_verts), 4+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[face_idx_z, 1+len(mesh_verts), 5+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[face_idx_z, 2+len(mesh_verts), 5+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[0+len(mesh_verts), 3+len(mesh_verts), 4+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[2+len(mesh_verts), 4+len(mesh_verts), 5+len(mesh_verts)]]).to(mesh_device))
    new_faces_list.append(torch.tensor([[1+len(mesh_verts), 3+len(mesh_verts), 5+len(mesh_verts)]]).to(mesh_device))
    hole_mesh_faces = torch.cat(new_faces_list, dim=0)
    hole_mesh = pytorch3d.structures.Meshes(verts=hole_mesh_verts.unsqueeze(0) , faces=hole_mesh_faces.unsqueeze(0))
    return hole_mesh
    

In [4]:
def create_hole_in_mesh(mesh, face_idx, num_new=1):
    mesh_verts = mesh.verts_packed().detach().clone()
    mesh_faces = mesh.faces_packed().detach().clone()
    mesh_device = mesh_verts.device
    
    face_idx_x = mesh_faces[face_idx][0].item()
    face_idx_y = mesh_faces[face_idx][1].item()
    face_idx_z = mesh_faces[face_idx][2].item()
    vertex_x = mesh_verts[face_idx_x]
    vertex_y = mesh_verts[face_idx_y]
    vertex_z = mesh_verts[face_idx_z]
    
    vertex_c = midpoint([vertex_x, vertex_y, vertex_z])
    vertex_cx = midpoint([vertex_c, vertex_x]).unsqueeze(0)
    vertex_cy = midpoint([vertex_c, vertex_y]).unsqueeze(0)
    vertex_cz = midpoint([vertex_c, vertex_z]).unsqueeze(0)
    
    new_verts_list = [mesh_verts, vertex_cx, vertex_cy, vertex_cz]
    new_faces_list = [mesh_faces[np.array([i for i in range(len(mesh_faces)) if i!= face_idx])]]
    hole_mesh_verts = torch.cat(new_verts_list, dim=0)
    
    for LL, LL_idx, UL, UL_idx, UR, UR_idx, LR, LR_idx in [[vertex_x.unsqueeze(0), face_idx_x, vertex_cx, len(mesh_verts), vertex_cy, len(mesh_verts)+1, vertex_y.unsqueeze(0), face_idx_y],
                                                           [vertex_y.unsqueeze(0), face_idx_y, vertex_cy, len(mesh_verts)+1, vertex_cz, len(mesh_verts)+2, vertex_z.unsqueeze(0), face_idx_z],
                                                           [vertex_z.unsqueeze(0), face_idx_z, vertex_cz, len(mesh_verts)+2, vertex_cx, len(mesh_verts), vertex_x.unsqueeze(0), face_idx_x]]:
        trap_verts_top = []
        trap_verts_bottom = []
        # creating vertices at top and bottom of trapezoid, by linear interpolation
        for alpha in list(np.linspace(0,1,num=num_new+2))[1:-1]:
            trap_verts_top.append((UL*(1-alpha)+UR*alpha))
        for alpha in list(np.linspace(0,1,num=num_new+3))[1:-1]:
            trap_verts_bottom.append((LL*(1-alpha)+LR*alpha))
        # creating faces from left to right
        trap_verts_interlaced = []
        top_i = 0
        bottom_i = 0
        for i in range((2*num_new)+1):
            if i%2 ==0:
                trap_verts_interlaced.append(trap_verts_bottom[bottom_i])
                bottom_i+=1
            else:
                trap_verts_interlaced.append(trap_verts_top[top_i])
                top_i+=1
        trap_idxs = [LL_idx, UL_idx] + list(range(len(hole_mesh_verts), len(hole_mesh_verts)+len(trap_verts_interlaced))) + [UR_idx, LR_idx]
        for i in range(len(trap_idxs)-2):
            new_faces_list.append(torch.tensor([[trap_idxs[i], trap_idxs[i+1], trap_idxs[i+2]]]).to(mesh_device))
        hole_mesh_verts = torch.cat([hole_mesh_verts]+trap_verts_interlaced, dim=0)
            
    hole_mesh_faces = torch.cat(new_faces_list, dim=0)
    hole_mesh = pytorch3d.structures.Meshes(verts=hole_mesh_verts.unsqueeze(0), faces=hole_mesh_faces.unsqueeze(0))
    return hole_mesh

In [6]:
#model_path = "notebooks/triangular_cube_8.obj"
model_path = "notebooks/triangular_mesh_cube.obj"
device = torch.device("cuda:0")
mesh = general_utils.load_untextured_mesh(model_path, device)

/opt/conda/envs/standard/lib/python3.6/site-packages/pytorch3d/io/obj_io.py:495: UserWarning: Mtl file does not exist: notebooks/./asdf.obj.mtl
  warnings.warn(f"Mtl file does not exist: {f_mtl}")


In [9]:
mesh_with_hole = create_hole_in_mesh(mesh, 1236, num_new=7)
save_obj("notebooks/hole.obj", mesh_with_hole.verts_packed(), mesh_with_hole.faces_packed())